In [1]:
import sys
import os
sys.path.append(os.path.abspath("Pytorch/"))
sys.path.append(os.path.abspath("models/"))

In [2]:
from XVA import *
from XVAFBSNNs import *

In [3]:
from FBSNNs import *
from CallOptionsBasket import *

In [4]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import time

M = 64  # number of trajectories (batch size)
N = 50  # number of time snapshots
D = 100 # number of dimensions
Mm = N ** (1/5)

layers = [D + 1] + 4 * [256] + [1]

Xi = np.array([1] * int(D))[None, :]
T = 1.0

"Available architectures"
mode = "Naisnet"  # FC and Naisnet are available
activation = "Sine"  # Sine, ReLU and Tanh are available
model = CallOptionsBasket(Xi, T, M, N, D, Mm, layers, mode, activation)

n_iter = 15 * 10**2
lr = 1e-3

In [5]:
model.load_model("models/100DCallOptionXVAPaper.pth")

In [8]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import time

M = 64  # number of trajectories (batch size)
N = 50  # number of time snapshots
D = 100 # number of dimensions
Mm = N ** (1/5)

layers = [D+1] + 4 * [256] + [1]

Xi = np.array([1] * int(D))[None, :]
T = 1.0

"Available architectures"
mode = "Naisnet"  # FC and Naisnet are available
activation = "Sine"  # Sine, ReLU and Tanh are available
xvamodel = XVA(Xi, T, M, N, D, Mm, layers, mode, activation, model)

n_iter = 15 * 10**2
lr = 1e-3

In [12]:
n_iter = 31 * 10**2
lr = 1e-5

In [13]:
tot = time.time()
print(xvamodel.device)
graph = xvamodel.train(n_iter, lr)
print("total time:", time.time() - tot, "s")

cuda:0
It: 1400, Loss: 2.205e-01, Y0: 0.065, Time: 0.41, Learning Rate: 1.000e-05
It: 1500, Loss: 1.717e-01, Y0: 0.038, Time: 60.45, Learning Rate: 1.000e-05
It: 1600, Loss: 1.429e-01, Y0: 0.037, Time: 35.79, Learning Rate: 1.000e-05
It: 1700, Loss: 1.568e-01, Y0: 0.037, Time: 35.11, Learning Rate: 1.000e-05
It: 1800, Loss: 1.479e-01, Y0: 0.039, Time: 42.16, Learning Rate: 1.000e-05
It: 1900, Loss: 1.464e-01, Y0: 0.036, Time: 63.03, Learning Rate: 1.000e-05
It: 2000, Loss: 1.449e-01, Y0: 0.038, Time: 37.99, Learning Rate: 1.000e-05
It: 2100, Loss: 1.426e-01, Y0: 0.044, Time: 35.14, Learning Rate: 1.000e-05
It: 2200, Loss: 1.206e-01, Y0: 0.038, Time: 41.65, Learning Rate: 1.000e-05
It: 2300, Loss: 1.133e-01, Y0: 0.042, Time: 56.67, Learning Rate: 1.000e-05
It: 2400, Loss: 1.187e-01, Y0: 0.041, Time: 51.65, Learning Rate: 1.000e-05
It: 2500, Loss: 1.131e-01, Y0: 0.042, Time: 35.42, Learning Rate: 1.000e-05
It: 2600, Loss: 1.105e-01, Y0: 0.036, Time: 35.49, Learning Rate: 1.000e-05
It: 27

In [7]:
xvamodel.load_model("models/100DCallOptionXVAHalfCollateralized.pth")

In [15]:
t_test, W_test, X_test, C_test = xvamodel.fetch_minibatch()
X_pred, Y_pred = xvamodel.predict(C_test, t_test, W_test, X_test)

if type(t_test).__module__ != 'numpy':
    t_test = t_test.cpu().numpy()
if type(X_pred).__module__ != 'numpy':
    X_pred = X_pred.cpu().detach().numpy()
if type(Y_pred).__module__ != 'numpy':
    Y_pred = Y_pred.cpu().detach().numpy()

for i in range(63):
    t_test_i, W_test_i, X_test_i, C_test_i = xvamodel.fetch_minibatch()
    X_pred_i, Y_pred_i = xvamodel.predict(C_test_i, t_test_i, W_test_i, X_test_i)
    if type(t_test_i).__module__ != 'numpy':
        t_test_i = t_test_i.cpu().numpy()
    if type(X_pred_i).__module__ != 'numpy':
        X_pred_i = X_pred_i.cpu().detach().numpy()
    if type(Y_pred_i).__module__ != 'numpy':
        Y_pred_i = Y_pred_i.cpu().detach().numpy()
    t_test = np.concatenate((t_test, t_test_i), axis=0)
    X_pred = np.concatenate((X_pred, X_pred_i), axis=0)
    Y_pred = np.concatenate((Y_pred, Y_pred_i), axis=0)

In [16]:
Y_preds = Y_pred.reshape(4096, 51)

In [17]:
Y_preds[0,:][0] * 100

4.1841428726911545

In [18]:
xvamodel.save_model("models/100DCallOptionXVAHalfCollateralized.pth")